In [3]:
from __future__ import print_function, division
from builtins import range
import os
import sys
import numpy as np
import pandas as pd

In [4]:
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, UpSampling1D
from keras.models import Model
from keras.models import model_from_json

c:\users\eratsau\appdata\local\programs\python\python36\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
# loading the final_data from txt file
final_data = np.loadtxt('final_data.txt', dtype=float)
#sqn_data = np.loadtxt('seq_ID.txt', dtype='str')
sqn_data = np.loadtxt('seq_ID.txt', dtype=bytes, delimiter="\n").astype(str)

In [6]:
sqn_data[:10]

array(['TUNE9CG3ZHRAYBFYtr', 'TUNE9CG3CDTCKVHKtr', 'TUNEYFGDFZTBSDPZtr',
       'TUNEYFGDUW3WSS5Ztr', 'TUNEYFGBEKUYYXKZtr', 'TUNDZ6Z2BCFB96T3tr',
       'TUNDZ6Z2YUJVF3UVtr', 'TUNE8UKQJA4PKNYUtr', 'TUNDXM4GMP79FKXYtr',
       'TUNDVRC9GBR4TQ4Ktr'], dtype='<U18')

In [7]:
final_data.shape

(20565, 16)

In [8]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")

Loaded model from disk


In [9]:
from keras import backend as K

In [10]:
# Extracting the weights of Reduced Dimmension from Layer3 of Auto-encoder

compressed_layer = 3
get_3rd_layer_output = K.function([loaded_model.layers[0].input], [loaded_model.layers[compressed_layer].output])
compressed = get_3rd_layer_output([final_data])[0]

In [11]:
print(compressed.shape)
print(final_data.shape)

(20565, 4)
(20565, 16)


In [12]:
print(final_data[:10])
print(compressed[:10])

[[1.000e+00 1.399e+03 8.020e+02 2.951e+03 0.000e+00 3.275e+03 3.316e+03
  0.000e+00 0.000e+00 4.689e+03 4.689e+03 0.000e+00 5.680e+03 5.864e+03
  5.874e+03 0.000e+00]
 [1.000e+00 1.399e+03 8.020e+02 2.951e+03 0.000e+00 3.276e+03 3.316e+03
  0.000e+00 0.000e+00 4.689e+03 4.689e+03 0.000e+00 5.680e+03 5.864e+03
  5.874e+03 0.000e+00]
 [2.000e+00 1.299e+03 4.690e+02 2.952e+03 0.000e+00 3.275e+03 3.317e+03
  0.000e+00 0.000e+00 4.690e+03 4.690e+03 0.000e+00 5.681e+03 5.865e+03
  0.000e+00 0.000e+00]
 [2.000e+00 1.299e+03 4.690e+02 2.952e+03 0.000e+00 3.277e+03 3.317e+03
  0.000e+00 0.000e+00 4.690e+03 4.690e+03 0.000e+00 5.681e+03 5.865e+03
  0.000e+00 0.000e+00]
 [2.000e+00 1.299e+03 3.790e+02 2.952e+03 0.000e+00 3.276e+03 3.318e+03
  0.000e+00 0.000e+00 4.690e+03 4.690e+03 0.000e+00 5.681e+03 5.865e+03
  0.000e+00 0.000e+00]
 [3.000e+00 1.295e+03 6.480e+02 2.953e+03 0.000e+00 3.275e+03 3.319e+03
  0.000e+00 0.000e+00 4.691e+03 4.691e+03 0.000e+00 5.682e+03 5.865e+03
  0.000e+00 0.000e+00

In [14]:
# Testing the distance function

from scipy.spatial import distance
print(distance.euclidean(compressed[2], compressed[1]))

398.6902160644531


In [50]:
# To create dictionary of Product IDs having Euclidean distance <10

print(compressed.shape[0])
dup_dict = {}
x=0
for i in compressed[:5]:
    dup_list = []
    #print(i)
    y=0
    for j in compressed[:100]:
        if (x != y):
            #print(i, j, x, y)
            dist = distance.euclidean(i,j)
            #print(dist)
            if dist <10:
                #print(sentence)
                dup_list.append([sqn_data[y], dist])
        y=y+1
    print(dup_list)
    #print(list(dup_list))
    dup = dict(zip([sqn_data[x]], [dup_list]))
    dup_dict.update(dup)    
    #print(dup_dict)
    x=x+1
    #print("echo")

20565
[ 6489.93    9716.218  15256.633   3430.4697]
[ 6489.93    9716.218  15256.633   3430.4697] [ 6490.274  9716.325 15257.132  3430.726] 0 1
0.6666648387908936
[ 6489.93    9716.218  15256.633   3430.4697] [ 6624.793  9930.821 15266.806  3122.901] 0 2
398.6787109375
[ 6489.93    9716.218  15256.633   3430.4697] [ 6625.4805  9931.04   15267.805   3123.4136] 0 3
398.6612243652344
[ 6489.93    9716.218  15256.633   3430.4697] [ 6648.8228  9989.973  15254.52    3116.458 ] 0 4
445.86590576171875
[ 6489.93    9716.218  15256.633   3430.4697] [ 6584.0786  9816.452  15296.714   3139.2708] 0 5
324.521484375
[ 6489.93    9716.218  15256.633   3430.4697] [ 6584.4224  9816.562  15297.211   3139.5273] 0 6
324.48681640625
[ 6489.93    9716.218  15256.633   3430.4697] [ 6328.13    9133.96   15468.168   3244.8306] 0 7
666.6427001953125
[ 6489.93    9716.218  15256.633   3430.4697] [ 6502.733  9611.332 15345.861  3160.838] 0 8
303.03131103515625
[ 6489.93    9716.218  15256.633   3430.4697] [ 6366.1

[ 6624.793  9930.821 15266.806  3122.901] [ 6633.7036  9897.136  15289.809   3140.9478] 2 26
45.48560333251953
[ 6624.793  9930.821 15266.806  3122.901] [ 6759.8384 10400.908  15134.942   3335.503 ] 2 27
549.3692016601562
[ 6624.793  9930.821 15266.806  3122.901] [ 6663.    10176.389 15190.517  3370.21 ] 2 28
358.8104248046875
[ 6624.793  9930.821 15266.806  3122.901] [ 6663.6855 10176.606  15191.516   3370.7227] 2 29
359.17572021484375
[ 6624.793  9930.821 15266.806  3122.901] [ 6723.0664 10147.508  15258.316   3098.609 ] 2 30
239.31753540039062
[ 6624.793  9930.821 15266.806  3122.901] [ 6740.099  10189.085  15254.842   3094.5564] 2 31
284.5033874511719
[ 6624.793  9930.821 15266.806  3122.901] [ 6741.3    10188.711  15255.722   3095.4402] 2 32
284.53094482421875
[ 6624.793  9930.821 15266.806  3122.901] [ 6561.9785  9844.147  15287.706   3434.5166] 2 33
330.1502685546875
[ 6624.793  9930.821 15266.806  3122.901] [ 6424.2075  9480.922  15370.739   3480.6062] 2 34
617.5753173828125
[ 

103.40380096435547
[ 6648.8228  9989.973  15254.52    3116.458 ] [ 6759.8384 10400.908  15134.942   3335.503 ] 4 27
493.4286804199219
[ 6648.8228  9989.973  15254.52    3116.458 ] [ 6663.    10176.389 15190.517  3370.21 ] 4 28
321.61834716796875
[ 6648.8228  9989.973  15254.52    3116.458 ] [ 6663.6855 10176.606  15191.516   3370.7227] 4 29
321.9830322265625
[ 6648.8228  9989.973  15254.52    3116.458 ] [ 6723.0664 10147.508  15258.316   3098.609 ] 4 30
175.1069793701172
[ 6648.8228  9989.973  15254.52    3116.458 ] [ 6740.099  10189.085  15254.842   3094.5564] 4 31
220.12921142578125
[ 6648.8228  9989.973  15254.52    3116.458 ] [ 6741.3    10188.711  15255.722   3095.4402] 4 32
220.209228515625
[ 6648.8228  9989.973  15254.52    3116.458 ] [ 6561.9785  9844.147  15287.706   3434.5166] 4 33
362.0352478027344
[ 6648.8228  9989.973  15254.52    3116.458 ] [ 6424.2075  9480.922  15370.739   3480.6062] 4 34
675.0523071289062
[ 6648.8228  9989.973  15254.52    3116.458 ] [ 6546.3184  9846.

In [60]:
# function to find and print duplicate product IDs

def find_dup(prodID):
    id = sqn_data.tolist().index(prodID)
    y=0
    i = compressed[id]
    for j in compressed:
        if (id != y):
            #print(i, j, x, y)
            dist = distance.euclidean(i,j)
            #print(dist)
            if dist <10:
                #sentence = (i for i in sqn_data[y])
                #print(sentence)
                dup_list.append([sqn_data[y], dist])
                #dup_dict[i].append(print(sqn_data[y]))
            #diff_mat[x][y] = dist
        y=y+1
    return dup_list

In [61]:
print(find_dup('TUNE9CG3CDTCKVHKtr'))

[['TUNEYFGBMTJZYXDZtr', 1.775009036064148], ['TUNEYFGBF5GRKZGPtr', 7.083601474761963], ['TUNE9CG3ZHRAYBFYtr', 0.6666648387908936]]
